# ETL on 311 Complaints dataset
## Note dimension tables CANNOT have null values 
## However, fact tables CAN have null values


# Loading in CSV File into Python/Google Colaboratory

# Previously the data had been filtered from the NYC Open Data Website to only include complaints relating to noise from 2016 onwards 

# The file was converted from an .xlsx file to a .csv file

# The formats "Created Date" and "Closed Date" columns were changed in to "Long Date" from "Short Date" 

In [ ]:
import pandas as pd
from google.cloud import bigquery # package allows us to access GCP platform project
import os

In [ ]:
! gdown --id 1olZAnsDfC2PgrQFPQ5RBNkhwVvB5h8Iv

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1olZAnsDfC2PgrQFPQ5RBNkhwVvB5h8Iv
To: /content/311_Service_Requests_from_2010_to_Present.csv
100% 63.0M/63.0M [00:00<00:00, 224MB/s]


In [ ]:
# CSV file of filtered data retrieved from Socratica API
df = pd.read_csv("311_Service_Requests_from_2010_to_Present.csv")
print(len(df))
df.head()

108586


,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,46367528,06/02/2020 06:09:00 PM,06/03/2020 07:00:00 PM,DEP,Department of Environmental Protection,Noise,"Noise, Ice Cream Truck (NR4)",NaN,11228,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.619700,-74.008054,"(40.619700300698476, -74.0080543255101)"
1,43207999,07/06/2019 01:03:33 AM,07/06/2019 04:48:42 AM,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10458,2779 BRIGGS AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.867763,-73.890405,"(40.86776277663766, -73.8904054107113)"
2,43211313,07/06/2019 07:49:10 PM,07/06/2019 11:24:59 PM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,NaN,10458,357 EAST 201 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.869586,-73.882632,"(40.86958584547678, -73.88263236892325)"
3,43223100,07/09/2019 12:20:14 AM,07/09/2019 06:55:43 AM,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,10458,2711 VALENTINE AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.866867,-73.892327,"(40.86686706926165, -73.89232671971808)"
4,43228977,07/08/2019 04:36:32 AM,07/08/2019 05:29:55 AM,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Store/Commercial,11372,95-15B NORTHERN BOULEVARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.756880,-73.872890,"(40.75687992090349, -73.87289029929983)"


In [ ]:
# Data Profiling: 
# Created a new df called master_columns, 
# Check the line 4 below to see if there are NaN values in that row
df.isna().any()

Unique Key                        False
Created Date                      False
Closed Date                        True
Agency                            False
Agency Name                       False
Complaint Type                    False
Descriptor                        False
Location Type                      True
Incident Zip                      False
Incident Address                   True
Street Name                        True
Cross Street 1                     True
Cross Street 2                     True
Intersection Street 1              True
Intersection Street 2              True
Address Type                       True
City                               True
Landmark                           True
Facility Type                      True
Status                            False
Due Date                           True
Resolution Description             True
Resolution Action Updated Date     True
Community Board                   False
BBL                                True


In [ ]:
# Adding Date Duplicate to the end of the dataframe
# Source: https://www.statology.org/pandas-create-duplicate-column/
# df['Date Duplicate'] = df.loc[:,'Created Date']
# df.head()

In [ ]:
# Adding the dataframe to a certain location
# https://discuss.codecademy.com/t/can-we-add-a-new-column-at-a-specific-position-in-a-pandas-dataframe/355842
#df.insert(2, 'Date', df.loc[:,'Created Date'])
# df.head()

,Unique Key,Created Date,Date,Created Date Short,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,46367528,"Tuesday, June 2, 2020","Tuesday, June 2, 2020",6/2/2020,"Wednesday, June 3, 2020",DEP,Department of Environmental Protection,Noise,"Noise, Ice Cream Truck (NR4)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.619700,-74.008054,"(40.619700300698476, -74.0080543255101)"
1,43207999,"Saturday, July 6, 2019","Saturday, July 6, 2019",7/6/2019,"Saturday, July 6, 2019",NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.867763,-73.890405,"(40.86776277663766, -73.8904054107113)"
2,43211313,"Saturday, July 6, 2019","Saturday, July 6, 2019",7/6/2019,"Saturday, July 6, 2019",NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.869586,-73.882632,"(40.86958584547678, -73.88263236892325)"
3,43223100,"Tuesday, July 9, 2019","Tuesday, July 9, 2019",7/9/2019,"Tuesday, July 9, 2019",NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.866867,-73.892327,"(40.86686706926165, -73.89232671971808)"
4,43228977,"Monday, July 8, 2019","Monday, July 8, 2019",7/8/2019,"Monday, July 8, 2019",NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Store/Commercial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.756880,-73.872890,"(40.75687992090349, -73.87289029929983)"


In [ ]:
master_columns =df[[ 'Created Date', 'Agency', 'Agency Name', 'Complaint Type', 'Location Type','Incident Zip', 
                    'Incident Address', 'Street Name', 'City', 'Status', 'Borough', 
'Open Data Channel Type']]
print(len(master_columns))
master_columns.head()



108586


,Created Date,Agency,Agency Name,Complaint Type,Location Type,Incident Zip,Incident Address,Street Name,City,Status,Borough,Open Data Channel Type
0,06/02/2020 06:09:00 PM,DEP,Department of Environmental Protection,Noise,NaN,11228,NaN,NaN,BROOKLYN,Closed,BROOKLYN,ONLINE
1,07/06/2019 01:03:33 AM,NYPD,New York City Police Department,Noise - Street/Sidewalk,Street/Sidewalk,10458,2779 BRIGGS AVENUE,BRIGGS AVENUE,BRONX,Closed,BRONX,MOBILE
2,07/06/2019 07:49:10 PM,NYPD,New York City Police Department,Noise - Residential,NaN,10458,357 EAST 201 STREET,EAST 201 STREET,BRONX,Closed,BRONX,PHONE
3,07/09/2019 12:20:14 AM,NYPD,New York City Police Department,Noise - Vehicle,Street/Sidewalk,10458,2711 VALENTINE AVENUE,VALENTINE AVENUE,BRONX,Closed,BRONX,MOBILE
4,07/08/2019 04:36:32 AM,NYPD,New York City Police Department,Noise - Commercial,Store/Commercial,11372,95-15B NORTHERN BOULEVARD,NORTHERN BOULEVARD,JACKSON HEIGHTS,Closed,QUEENS,ONLINE


In [ ]:
master_columns.isna().any()

Created Date              False
Agency                    False
Agency Name               False
Complaint Type            False
Location Type              True
Incident Zip              False
Incident Address           True
Street Name                True
City                       True
Status                    False
Borough                   False
Open Data Channel Type    False
dtype: bool

In [ ]:
# Filling the columns that are known to have NaN values and filling them with 
# 'Not Applicable' will remove all remaining NaN values from the dataframe

# The borough column will always have a value according to the data profiling done
# Most Cities in New York City share the same name as their borough (for the exception of Queens)
# So fill NaN values from the City column with the name of the borough
master_columns[['City']] = master_columns[['City']].fillna(master_columns[['Borough']]) 

print(len(master_columns))
master_columns.head()

108586


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Created Date,Agency,Agency Name,Complaint Type,Location Type,Incident Zip,Incident Address,Street Name,City,Status,Borough,Open Data Channel Type
0,06/02/2020 06:09:00 PM,DEP,Department of Environmental Protection,Noise,NaN,11228,NaN,NaN,BROOKLYN,Closed,BROOKLYN,ONLINE
1,07/06/2019 01:03:33 AM,NYPD,New York City Police Department,Noise - Street/Sidewalk,Street/Sidewalk,10458,2779 BRIGGS AVENUE,BRIGGS AVENUE,BRONX,Closed,BRONX,MOBILE
2,07/06/2019 07:49:10 PM,NYPD,New York City Police Department,Noise - Residential,NaN,10458,357 EAST 201 STREET,EAST 201 STREET,BRONX,Closed,BRONX,PHONE
3,07/09/2019 12:20:14 AM,NYPD,New York City Police Department,Noise - Vehicle,Street/Sidewalk,10458,2711 VALENTINE AVENUE,VALENTINE AVENUE,BRONX,Closed,BRONX,MOBILE
4,07/08/2019 04:36:32 AM,NYPD,New York City Police Department,Noise - Commercial,Store/Commercial,11372,95-15B NORTHERN BOULEVARD,NORTHERN BOULEVARD,JACKSON HEIGHTS,Closed,QUEENS,ONLINE


# ETL on 311 Data

# Creating a dataframe for the Location Dimension Table and Cleaning/ Transforming it 

In [ ]:
location_dim_tbl = master_columns[['Location Type', 'Incident Zip', 'Incident Address', 'Street Name', 'City', 'Borough']]
print(len(location_dim_tbl))
location_dim_tbl.head()

108586


,Location Type,Incident Zip,Incident Address,Street Name,City,Borough
0,NaN,11228,NaN,NaN,BROOKLYN,BROOKLYN
1,Street/Sidewalk,10458,2779 BRIGGS AVENUE,BRIGGS AVENUE,BRONX,BRONX
2,NaN,10458,357 EAST 201 STREET,EAST 201 STREET,BRONX,BRONX
3,Street/Sidewalk,10458,2711 VALENTINE AVENUE,VALENTINE AVENUE,BRONX,BRONX
4,Store/Commercial,11372,95-15B NORTHERN BOULEVARD,NORTHERN BOULEVARD,JACKSON HEIGHTS,QUEENS


In [ ]:
# We need to rename the column names to have underscores as required for columns in GBQ
location_dim_tbl = location_dim_tbl.rename(columns = {'Location Type':'Location_Type', 'Incident Zip': 'Incident_Zip', 'Incident Address':'Incident_Address', 'Street Name': 'Street_Name'})
print(len(location_dim_tbl))
location_dim_tbl.head()

108586


,Location_Type,Incident_Zip,Incident_Address,Street_Name,City,Borough
0,NaN,11228,NaN,NaN,BROOKLYN,BROOKLYN
1,Street/Sidewalk,10458,2779 BRIGGS AVENUE,BRIGGS AVENUE,BRONX,BRONX
2,NaN,10458,357 EAST 201 STREET,EAST 201 STREET,BRONX,BRONX
3,Street/Sidewalk,10458,2711 VALENTINE AVENUE,VALENTINE AVENUE,BRONX,BRONX
4,Store/Commercial,11372,95-15B NORTHERN BOULEVARD,NORTHERN BOULEVARD,JACKSON HEIGHTS,QUEENS


In [ ]:
# Creating the composite key, which will temporarily server as the surrogate key for the data
#location_dim_tbl.insert(0,'Location_Dim_ID', range(1, 1+ len(location_dim_tbl)))

# Concatenating the columns which are crucial for distinguishing a row entry 
# How to combine two columns of text: https://sparkbyexamples.com/pandas/pandas-combine-two-columns-of-text-in-dataframe/

location_dim_tbl.insert(0,'Location_Dim_ID', location_dim_tbl['Incident_Address'] + ', ' + location_dim_tbl['City'] + ', NEW YORK' + ', ' + location_dim_tbl['Incident_Zip'].astype(str))


print(len(location_dim_tbl)) # It turns out each row is unique, no duplicates
location_dim_tbl.head()

108586


,Location_Dim_ID,Location_Type,Incident_Zip,Incident_Address,Street_Name,City,Borough
0,NaN,NaN,11228,NaN,NaN,BROOKLYN,BROOKLYN
1,"2779 BRIGGS AVENUE, BRONX, NEW YORK, 10458",Street/Sidewalk,10458,2779 BRIGGS AVENUE,BRIGGS AVENUE,BRONX,BRONX
2,"357 EAST 201 STREET, BRONX, NEW YORK, 10458",NaN,10458,357 EAST 201 STREET,EAST 201 STREET,BRONX,BRONX
3,"2711 VALENTINE AVENUE, BRONX, NEW YORK, 10458",Street/Sidewalk,10458,2711 VALENTINE AVENUE,VALENTINE AVENUE,BRONX,BRONX
4,"95-15B NORTHERN BOULEVARD, JACKSON HEIGHTS, NE...",Store/Commercial,11372,95-15B NORTHERN BOULEVARD,NORTHERN BOULEVARD,JACKSON HEIGHTS,QUEENS


In [ ]:
# Create copy of dimension table, and drop duplicates. This dataframe will 
# be loaded in as Location Dimension Table 
location_dim_tbl2 = location_dim_tbl.copy()
location_dim_tbl2 = location_dim_tbl2.drop_duplicates()
print(len(location_dim_tbl2))
location_dim_tbl2.head()

19265


,Location_Dim_ID,Location_Type,Incident_Zip,Incident_Address,Street_Name,City,Borough
0,NaN,NaN,11228,NaN,NaN,BROOKLYN,BROOKLYN
1,"2779 BRIGGS AVENUE, BRONX, NEW YORK, 10458",Street/Sidewalk,10458,2779 BRIGGS AVENUE,BRIGGS AVENUE,BRONX,BRONX
2,"357 EAST 201 STREET, BRONX, NEW YORK, 10458",NaN,10458,357 EAST 201 STREET,EAST 201 STREET,BRONX,BRONX
3,"2711 VALENTINE AVENUE, BRONX, NEW YORK, 10458",Street/Sidewalk,10458,2711 VALENTINE AVENUE,VALENTINE AVENUE,BRONX,BRONX
4,"95-15B NORTHERN BOULEVARD, JACKSON HEIGHTS, NE...",Store/Commercial,11372,95-15B NORTHERN BOULEVARD,NORTHERN BOULEVARD,JACKSON HEIGHTS,QUEENS


In [ ]:
#location_dim_tbl = location_dim_tbl.drop('Complete_Address', axis=1)
print(len(location_dim_tbl))
location_dim_tbl.head()

108586


,Location_Dim_ID,Location_Type,Incident_Zip,Incident_Address,Street_Name,City,Borough
0,NaN,NaN,11228,NaN,NaN,BROOKLYN,BROOKLYN
1,"2779 BRIGGS AVENUE, BRONX, NEW YORK, 10458",Street/Sidewalk,10458,2779 BRIGGS AVENUE,BRIGGS AVENUE,BRONX,BRONX
2,"357 EAST 201 STREET, BRONX, NEW YORK, 10458",NaN,10458,357 EAST 201 STREET,EAST 201 STREET,BRONX,BRONX
3,"2711 VALENTINE AVENUE, BRONX, NEW YORK, 10458",Street/Sidewalk,10458,2711 VALENTINE AVENUE,VALENTINE AVENUE,BRONX,BRONX
4,"95-15B NORTHERN BOULEVARD, JACKSON HEIGHTS, NE...",Store/Commercial,11372,95-15B NORTHERN BOULEVARD,NORTHERN BOULEVARD,JACKSON HEIGHTS,QUEENS


In [ ]:
# Creating the table_id for this dataset for GBQ
# The hierarchy of how things are named in Google Big Query 
dataset_name = 'complaints'
table_id_loc_dim = dataset_name + '.' + 'location_dim_tbl2'

# Creating a dataframe for the Complaint Channel Dimension Table and Cleaning/ Transforming it 

In [ ]:
complaint_channel_dim_tbl = master_columns[['Open Data Channel Type']]
print(len(complaint_channel_dim_tbl))
complaint_channel_dim_tbl.head()

108586


,Open Data Channel Type
0,ONLINE
1,MOBILE
2,PHONE
3,MOBILE
4,ONLINE


In [ ]:
# We need to rename the column names to have underscores as required for columns in GBQ
complaint_channel_dim_tbl = complaint_channel_dim_tbl.rename(columns = {'Open Data Channel Type':'Open_Data_Channel_Type' })

print(len(complaint_channel_dim_tbl))
complaint_channel_dim_tbl.head()



108586


,Open_Data_Channel_Type
0,ONLINE
1,MOBILE
2,PHONE
3,MOBILE
4,ONLINE


In [ ]:
complaint_channel_dim_tbl2= complaint_channel_dim_tbl.copy() 
complaint_channel_dim_tbl2 = complaint_channel_dim_tbl2.drop_duplicates()
print(len(complaint_channel_dim_tbl2))
complaint_channel_dim_tbl2.head()

4


,Open_Data_Channel_Type
0,ONLINE
1,MOBILE
2,PHONE
6461,UNKNOWN


In [ ]:
# Creating the surrogate key
complaint_channel_dim_tbl2.insert(0,'Complaint_Channel_ID', range(1, 1+ len(complaint_channel_dim_tbl2)))
print(len((complaint_channel_dim_tbl2)))
complaint_channel_dim_tbl2.head()

4


,Complaint_Channel_ID,Open_Data_Channel_Type
0,1,ONLINE
1,2,MOBILE
2,3,PHONE
6461,4,UNKNOWN


In [ ]:
# # Creating the table_id for this dataset for GBQ
# The hierarchy of how things are named in Google Big Query 
dataset_name = 'complaints'
table_id_odcc = dataset_name + '.' + 'complaint_channel_dim_tbl2'

# Creating a dataframe for the Agency Dimension Table and Cleaning/ Transforming it 

In [ ]:
agency_dim_tbl = master_columns[['Agency', 'Agency Name']]
print(len(agency_dim_tbl))
agency_dim_tbl.head()

108586


,Agency,Agency Name
0,DEP,Department of Environmental Protection
1,NYPD,New York City Police Department
2,NYPD,New York City Police Department
3,NYPD,New York City Police Department
4,NYPD,New York City Police Department


In [ ]:
# We need to rename the column names to have underscores as required for columns in GBQ
agency_dim_tbl = agency_dim_tbl.rename(columns = {'Agency Name': 'Agency_Name'})
agency_dim_tbl.head()

,Agency,Agency_Name
0,DEP,Department of Environmental Protection
1,NYPD,New York City Police Department
2,NYPD,New York City Police Department
3,NYPD,New York City Police Department
4,NYPD,New York City Police Department


In [ ]:
# Creating the surrogate key
agency_dim_tbl.insert(0,'Agency_Dim_ID', agency_dim_tbl['Agency'] + ' - ' + agency_dim_tbl['Agency_Name'])
print(len((agency_dim_tbl)))
agency_dim_tbl.head()


108586


,Agency_Dim_ID,Agency,Agency_Name
0,DEP - Department of Environmental Protection,DEP,Department of Environmental Protection
1,NYPD - New York City Police Department,NYPD,New York City Police Department
2,NYPD - New York City Police Department,NYPD,New York City Police Department
3,NYPD - New York City Police Department,NYPD,New York City Police Department
4,NYPD - New York City Police Department,NYPD,New York City Police Department


In [ ]:
agency_dim_tbl2 = agency_dim_tbl.copy()
agency_dim_tbl2 = agency_dim_tbl2.drop_duplicates()
print(len(agency_dim_tbl2))
agency_dim_tbl2.head()

4


,Agency_Dim_ID,Agency,Agency_Name
0,DEP - Department of Environmental Protection,DEP,Department of Environmental Protection
1,NYPD - New York City Police Department,NYPD,New York City Police Department
747,EDC - Economic Development Corporation,EDC,Economic Development Corporation
44963,DSNY - Department of Sanitation,DSNY,Department of Sanitation


In [ ]:
# # Creating the table_id for this dataset for GBQ
# The hierarchy of how things are named in Google Big Query 
dataset_name = 'complaints'
table_id_agency = dataset_name + '.' + 'agency_dim_tbl2'

# Creating a dataframe for the Complaint Type Dimension Table and Cleaning/ Transforming it 

In [ ]:
complaint_type_dim_tbl = master_columns[['Complaint Type']]
print(len(complaint_type_dim_tbl))
complaint_type_dim_tbl.head()

108586


,Complaint Type
0,Noise
1,Noise - Street/Sidewalk
2,Noise - Residential
3,Noise - Vehicle
4,Noise - Commercial


In [ ]:
# We need to rename the column names to have underscores as required for columns in GBQ
complaint_type_dim_tbl = complaint_type_dim_tbl.rename(columns = {'Complaint Type': 'Complaint_Type'})
print(len(complaint_type_dim_tbl))
complaint_type_dim_tbl.head()

108586


,Complaint_Type
0,Noise
1,Noise - Street/Sidewalk
2,Noise - Residential
3,Noise - Vehicle
4,Noise - Commercial


In [ ]:
complaint_type_dim_tbl2 = complaint_type_dim_tbl.copy()
complaint_type_dim_tbl2 = complaint_type_dim_tbl.drop_duplicates()
print(len(complaint_type_dim_tbl2))
complaint_type_dim_tbl2.head()

9


,Complaint_Type
0,Noise
1,Noise - Street/Sidewalk
2,Noise - Residential
3,Noise - Vehicle
4,Noise - Commercial


In [ ]:
# Creating the surrogate key
complaint_type_dim_tbl2.insert(0,'Complaint_Type_ID', range(1, 1+ len(complaint_type_dim_tbl2)))
print(len((complaint_type_dim_tbl2)))
complaint_type_dim_tbl2.head()

9


,Complaint_Type_ID,Complaint_Type
0,1,Noise
1,2,Noise - Street/Sidewalk
2,3,Noise - Residential
3,4,Noise - Vehicle
4,5,Noise - Commercial


In [ ]:
# # Creating the table_id for this dataset for GBQ
# The hierarchy of how things are named in Google Big Query 
dataset_name = 'complaints'
table_id_comp_type = dataset_name + '.' + 'complaint_type_dim_tbl2'

# Creating a dataframe for the Complaint Status Dimension Table and Cleaning/ Transforming it 

In [ ]:
complaint_status_dim_tbl = master_columns[['Status']]
print(len(complaint_status_dim_tbl))
complaint_status_dim_tbl.head()

108586


,Status
0,Closed
1,Closed
2,Closed
3,Closed
4,Closed


In [ ]:
# We need to rename the column names to have underscores as required for columns in GBQ
complaint_status_dim_tbl = complaint_status_dim_tbl.rename(columns = {'Status': 'Complaint_Status'})
print(len(complaint_status_dim_tbl))
complaint_status_dim_tbl.head()


108586


,Complaint_Status
0,Closed
1,Closed
2,Closed
3,Closed
4,Closed


In [ ]:
complaint_status_dim_tbl2 = complaint_status_dim_tbl.copy()
complaint_status_dim_tbl2 = complaint_status_dim_tbl.drop_duplicates()
print(len(complaint_status_dim_tbl2))
complaint_status_dim_tbl2.head()

7


,Complaint_Status
0,Closed
4567,Started
12019,Pending
13274,Open
16884,In Progress


In [ ]:
# Creating the surrogate key
complaint_status_dim_tbl2.insert(0,'Complaint_Status_ID', range(1, 1+ len(complaint_status_dim_tbl2)))
print(len((complaint_status_dim_tbl2)))
complaint_status_dim_tbl2.head()

7


,Complaint_Status_ID,Complaint_Status
0,1,Closed
4567,2,Started
12019,3,Pending
13274,4,Open
16884,5,In Progress


In [ ]:
# # Creating the table_id for this dataset for GBQ
# The hierarchy of how things are named in Google Big Query 
dataset_name = 'complaints'
table_id_comp_status = dataset_name + '.' + 'complaint_status_dim_tbl2'

# Creating a dataframe for the Date Dimension Table and Cleaning/ Transforming it 

In [ ]:
date_dim_tbl = master_columns[['Created Date']]

# We need to rename the column names to have underscores as required for columns in GBQ
date_dim_tbl = date_dim_tbl.rename(columns = {'Created Date': 'Created_Date'})

print(len(date_dim_tbl))
date_dim_tbl.head()

108586


,Created_Date
0,06/02/2020 06:09:00 PM
1,07/06/2019 01:03:33 AM
2,07/06/2019 07:49:10 PM
3,07/09/2019 12:20:14 AM
4,07/08/2019 04:36:32 AM


In [ ]:
# splitting the contents of a column using the .split method: https://stackoverflow.com/questions/37600711/pandas-split-column-into-multiple-columns-by-comma
date_slice = master_columns['Created Date'].str.split(' ', expand = True)
date_slice.head()

,0,1,2
0,06/02/2020,06:09:00,PM
1,07/06/2019,01:03:33,AM
2,07/06/2019,07:49:10,PM
3,07/09/2019,12:20:14,AM
4,07/08/2019,04:36:32,AM


In [ ]:
# splitting the contents of a column using the .split method: https://stackoverflow.com/questions/37600711/pandas-split-column-into-multiple-columns-by-comma
date_slice2 = date_slice[0].str.split('/', expand = True)

# How to convert the values of an dataframe column to int: 
# https://sparkbyexamples.com/pandas/pandas-convert-column-to-int/#:~:text=to%20int%20(Integer)-,Use%20pandas%20DataFrame.,int64%20%2C%20numpy.

day_nums = date_slice2[1].astype('int')
date_slice2.head()

,0,1,2
0,06,02,2020
1,07,06,2019
2,07,06,2019
3,07,09,2019
4,07,08,2019


In [ ]:
date_slice[0] = pd.to_datetime(date_slice[0])
date_slice[1]= pd.to_datetime(date_slice[1]) # this is really the format of the data
date_slice[0] = date_slice[0].copy()
date_slice[0] = pd.to_datetime(date_slice[0])

#date_slice = date_slice.drop(2, axis=1)

date_slice.head()

,0,1,2
0,2020-06-02,2023-01-25 06:09:00,PM
1,2019-07-06,2023-01-25 01:03:33,AM
2,2019-07-06,2023-01-25 07:49:10,PM
3,2019-07-09,2023-01-25 12:20:14,AM
4,2019-07-08,2023-01-25 04:36:32,AM


In [ ]:
# Source: https://www.geeksforgeeks.org/get-month-and-year-from-date-in-pandas-python/
year_num = date_slice[0].dt.year
year_int = year_num.astype('int')
year_int


0         2020
1         2019
2         2019
3         2019
4         2019
          ... 
108581    2022
108582    2022
108583    2022
108584    2022
108585    2022
Name: 0, Length: 108586, dtype: int64

In [ ]:
date_dim_tbl.insert(1, 'Date', date_slice[0])

# How to get the name of a month give a date (in datetime format):
# Source: https://pynative.com/python-get-month-name-from-number/#:~:text=Use%20the%20dt.,the%20month%20name%20in%20pandas.
date_dim_tbl.insert(2, 'Month', date_slice[0].dt.month_name())

# How to get the name of a day give a date (in datetime format):
# Source: https://www.geeksforgeeks.org/get-day-from-date-in-pandas-python/
date_dim_tbl.insert(3, 'Day_Of_Week', date_slice[0].dt.day_name() ) # date_slice[0].dt.dayofweek 

# The following commented code was used to create the variable day_nums
# in the cell above: 
# date_slice2 = date_slice[0].str.split('/', expand = True)
# How to convert the values of an dataframe column to int: 
# https://sparkbyexamples.com/pandas/pandas-convert-column-to-int/#:~:text=to%20int%20(Integer)-,Use%20pandas%20DataFrame.,int64%20%2C%20numpy.
# day_nums = date_slice2[1].astype('int')
date_dim_tbl.insert(4, 'Day_Number', day_nums)

# How to get the name of a year give a date (in datetime format):
# The following commented code was used to create the variable day_nums
# in the cell above: 
# Source: https://www.geeksforgeeks.org/get-month-and-year-from-date-in-pandas-python/
# year_num = date_slice[0].dt.year
# year_int = year_num.astype('int')
date_dim_tbl.insert(5, 'Year', year_int)

date_dim_tbl.insert(6, 'Time', date_slice[1])

#date_dim_tbl = date_slice.drop(0, axis=1) # Causes everything to get deleted
date_dim_tbl.head()

,Created_Date,Date,Month,Day_Of_Week,Day_Number,Year,Time
0,06/02/2020 06:09:00 PM,2020-06-02,June,Tuesday,2,2020,2023-01-25 06:09:00
1,07/06/2019 01:03:33 AM,2019-07-06,July,Saturday,6,2019,2023-01-25 01:03:33
2,07/06/2019 07:49:10 PM,2019-07-06,July,Saturday,6,2019,2023-01-25 07:49:10
3,07/09/2019 12:20:14 AM,2019-07-09,July,Tuesday,9,2019,2023-01-25 12:20:14
4,07/08/2019 04:36:32 AM,2019-07-08,July,Monday,8,2019,2023-01-25 04:36:32


In [ ]:
# Creating the composite key
# date_dim_tbl['Date_And_Time'] = date_dim_tbl['Date'] + ' - ' + date_dim_tbl['Time']
# Don't need this variable as "Created_Date" has date and time
# date_dim_tbl = date_dim_tbl.drop_duplicates()

# Created date already gives both date and time, which are critical to distinguishing
# Unique rows
date_dim_tbl = date_dim_tbl.rename(columns = {'Created_Date': 'Date_Dim_ID'})

print(len(date_dim_tbl))
date_dim_tbl.head()

108586


,Date_Dim_ID,Date,Month,Day_Of_Week,Day_Number,Year,Time
0,06/02/2020 06:09:00 PM,2020-06-02,June,Tuesday,2,2020,2023-01-25 06:09:00
1,07/06/2019 01:03:33 AM,2019-07-06,July,Saturday,6,2019,2023-01-25 01:03:33
2,07/06/2019 07:49:10 PM,2019-07-06,July,Saturday,6,2019,2023-01-25 07:49:10
3,07/09/2019 12:20:14 AM,2019-07-09,July,Tuesday,9,2019,2023-01-25 12:20:14
4,07/08/2019 04:36:32 AM,2019-07-08,July,Monday,8,2019,2023-01-25 04:36:32


In [ ]:
date_dim_tbl2 = date_dim_tbl.copy()
date_dim_tbl2 = date_dim_tbl2.drop_duplicates()
print(len(date_dim_tbl2))
date_dim_tbl2.head()

108408


,Date_Dim_ID,Date,Month,Day_Of_Week,Day_Number,Year,Time
0,06/02/2020 06:09:00 PM,2020-06-02,June,Tuesday,2,2020,2023-01-25 06:09:00
1,07/06/2019 01:03:33 AM,2019-07-06,July,Saturday,6,2019,2023-01-25 01:03:33
2,07/06/2019 07:49:10 PM,2019-07-06,July,Saturday,6,2019,2023-01-25 07:49:10
3,07/09/2019 12:20:14 AM,2019-07-09,July,Tuesday,9,2019,2023-01-25 12:20:14
4,07/08/2019 04:36:32 AM,2019-07-08,July,Monday,8,2019,2023-01-25 04:36:32


In [ ]:
# # Creating the table_id for this dataset for GBQ
# The hierarchy of how things are named in Google Big Query 
dataset_name = 'complaints'
table_id_date = dataset_name + '.' + 'date_dim_tbl2'

# Creating the Fact Table 

In [ ]:
complaint_fact_tbl = pd.DataFrame(agency_dim_tbl['Agency_Dim_ID'], columns = ['Agency_Dim_ID'])

In [ ]:
complaint_fact_tbl.insert(0,'Unique_Key', range(1, 1+ len(complaint_fact_tbl)))
complaint_fact_tbl.insert(2,'Complaint_Type', complaint_type_dim_tbl['Complaint_Type'])
complaint_fact_tbl.insert(3,'Complaint_Channel', complaint_channel_dim_tbl['Open_Data_Channel_Type'])
complaint_fact_tbl.insert(4,'Complaint_Status', complaint_status_dim_tbl['Complaint_Status'])
complaint_fact_tbl.insert(5,'Date', date_dim_tbl['Date'])
complaint_fact_tbl.insert(6,'Location_Dim_ID', location_dim_tbl['Location_Dim_ID'])

In [ ]:
print(len(complaint_fact_tbl))
complaint_fact_tbl.head()

108586


,Unique_Key,Agency_Dim_ID,Complaint_Type,Complaint_Channel,Complaint_Status,Date,Location_Dim_ID
0,1,DEP - Department of Environmental Protection,Noise,ONLINE,Closed,2020-06-02,NaN
1,2,NYPD - New York City Police Department,Noise - Street/Sidewalk,MOBILE,Closed,2019-07-06,"2779 BRIGGS AVENUE, BRONX, NEW YORK, 10458"
2,3,NYPD - New York City Police Department,Noise - Residential,PHONE,Closed,2019-07-06,"357 EAST 201 STREET, BRONX, NEW YORK, 10458"
3,4,NYPD - New York City Police Department,Noise - Vehicle,MOBILE,Closed,2019-07-09,"2711 VALENTINE AVENUE, BRONX, NEW YORK, 10458"
4,5,NYPD - New York City Police Department,Noise - Commercial,ONLINE,Closed,2019-07-08,"95-15B NORTHERN BOULEVARD, JACKSON HEIGHTS, NE..."


In [ ]:
# # Creating the table_id for this dataset for GBQ
# The hierarchy of how things are named in Google Big Query 
dataset_name = 'complaints'
table_id_311_fact = dataset_name + '.' + 'complaint_fact_tbl'

# Reading/Writing into Google Cloud project

In [ ]:
# Feeding in Json file generated from GBQ:
# How to generate the key file: 
# http://holowczak.com/creating-a-service-account-and-key-file-for-google-bigquery/3/?doing_wp_cron=1671419520.2419290542602539062500
# Uploading the JSON files is the cell below

In [ ]:
! gdown --id 1BFPm2cptd9EWD_187KSlYQrMUVy9sNzf

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1BFPm2cptd9EWD_187KSlYQrMUVy9sNzf
To: /content/cis4400proj-370416-74e9d8391c22.json
100% 2.32k/2.32k [00:00<00:00, 1.32MB/s]


In [ ]:
# Reading/Writing into Google Cloud project
# https://data.cityofnewyork.us/resource/erm2-nwe9.json

# Setting up the environment variable
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'cis4400proj-370416-74e9d8391c22.json'
# How to generate the JSON key file: 
# http://holowczak.com/creating-a-service-account-and-key-file-for-google-bigquery/3/?doing_wp_cron=1671419520.2419290542602539062500

client = bigquery.Client()

# Creating a dataframe for the Agency Dimension Table and Cleaning/ Transforming it 

# Loading data into Google Big Query

# Loading Location Dimension Table 

In [ ]:
# Client Object: How you read and write data into Google Big Query 
client = bigquery.Client()

In [ ]:
job_config = bigquery.LoadJobConfig(
    
  # Specify a partial schema. All columns are always written to the table
  # The schema is used to assist in data type definitions

  schema = [
      
      bigquery.SchemaField("Location_Dim_ID", bigquery.enums.SqlTypeNames.STRING),
      bigquery.SchemaField("Location_Type", bigquery.enums.SqlTypeNames.STRING),
      bigquery.SchemaField("Incident_Zip", bigquery.enums.SqlTypeNames.INTEGER),
      bigquery.SchemaField("Incident_Address", bigquery.enums.SqlTypeNames.STRING),
      bigquery.SchemaField("Street_Name", bigquery.enums.SqlTypeNames.STRING),  
      bigquery.SchemaField("City", bigquery.enums.SqlTypeNames.STRING),
      bigquery.SchemaField("Borough", bigquery.enums.SqlTypeNames.STRING)  
  ],

  write_disposition = "WRITE_TRUNCATE",

)

In [ ]:
job = client.load_table_from_dataframe(
    location_dim_tbl2, table_id_loc_dim, job_config= job_config
)
job.result()

LoadJob<project=cis4400proj-370416, location=US, id=2af1077f-23f4-41e1-8d81-8c74d65b3ceb>

# Loading Complaint Channel Dimension Table 

In [ ]:
# Client Object: How you read and write data into Google Big Query 
client = bigquery.Client()

In [ ]:
job_config = bigquery.LoadJobConfig(
    
  # Specify a partial schema. All columns are always written to the table
  # The schema is used to assist in data type definitions

  schema = [
      
      bigquery.SchemaField("Complaint_Channel_ID", bigquery.enums.SqlTypeNames.INTEGER),
      bigquery.SchemaField("Open_Data_Channel_Type", bigquery.enums.SqlTypeNames.STRING)
  ],

  write_disposition = "WRITE_TRUNCATE",

)

In [ ]:
job = client.load_table_from_dataframe(
    complaint_channel_dim_tbl2, table_id_odcc, job_config= job_config
)
job.result()

LoadJob<project=cis4400proj-370416, location=US, id=63ffd73d-c02c-4fe0-923c-9ec374ad66b3>

# Loading Agency Dimension Table 

In [ ]:
# Client Object: How you read and write data into Google Big Query 
client = bigquery.Client()

In [ ]:
job_config = bigquery.LoadJobConfig(
    
  # Specify a partial schema. All columns are always written to the table
  # The schema is used to assist in data type definitions

  schema = [
      
      bigquery.SchemaField("Agency_Dim_ID", bigquery.enums.SqlTypeNames.STRING),
      bigquery.SchemaField("Agency", bigquery.enums.SqlTypeNames.STRING), 
      bigquery.SchemaField("Agency_Name", bigquery.enums.SqlTypeNames.STRING),
  ],

  write_disposition = "WRITE_TRUNCATE",

)

In [ ]:
job = client.load_table_from_dataframe(
    agency_dim_tbl2, table_id_agency, job_config= job_config
)
job.result()

LoadJob<project=cis4400proj-370416, location=US, id=7530049f-c998-4e28-809d-4edaf17ec498>

# Loading Complaint Type Dimension Table




In [ ]:
# Client Object: How you read and write data into Google Big Query 
client = bigquery.Client()

In [ ]:
job_config = bigquery.LoadJobConfig(
    
  # Specify a partial schema. All columns are always written to the table
  # The schema is used to assist in data type definitions

  schema = [
      
      bigquery.SchemaField("Complaint_Type_ID", bigquery.enums.SqlTypeNames.INTEGER),
      bigquery.SchemaField("Complaint_Type", bigquery.enums.SqlTypeNames.STRING) 
  ],

  write_disposition = "WRITE_TRUNCATE",

)

In [ ]:
job = client.load_table_from_dataframe(
    complaint_type_dim_tbl2, table_id_comp_type, job_config= job_config
)
job.result()

LoadJob<project=cis4400proj-370416, location=US, id=f1c59194-8dab-4471-bb71-5ec0303c423a>

# Loading Complaint Status Dimension Table




In [ ]:
# Client Object: How you read and write data into Google Big Query 
client = bigquery.Client()

In [ ]:
job_config = bigquery.LoadJobConfig(
    
  # Specify a partial schema. All columns are always written to the table
  # The schema is used to assist in data type definitions

  schema = [
      
      bigquery.SchemaField("Complaint_Status_ID", bigquery.enums.SqlTypeNames.INTEGER),
      bigquery.SchemaField("Complaint_Status", bigquery.enums.SqlTypeNames.STRING) 
  ],

  write_disposition = "WRITE_TRUNCATE",

)

In [ ]:
job = client.load_table_from_dataframe(
    complaint_status_dim_tbl2, table_id_comp_status, job_config= job_config
)
job.result()

LoadJob<project=cis4400proj-370416, location=US, id=568d47ad-28f9-4052-a8c7-f9c15d5e80a6>

# Loading Date Dimension Table




In [ ]:
# Client Object: How you read and write data into Google Big Query 
client = bigquery.Client()

In [ ]:
job_config = bigquery.LoadJobConfig(
    
  # Specify a partial schema. All columns are always written to the table
  # The schema is used to assist in data type definitions

  # Converting all the date values to string works 

  schema = [
      
      bigquery.SchemaField("Date_Dim_ID", bigquery.enums.SqlTypeNames.STRING),
      bigquery.SchemaField("Date", bigquery.enums.SqlTypeNames.DATE),
      bigquery.SchemaField("Month", bigquery.enums.SqlTypeNames.STRING),
      bigquery.SchemaField("Year", bigquery.enums.SqlTypeNames.INTEGER),
      bigquery.SchemaField("Day_Of_Week", bigquery.enums.SqlTypeNames.STRING),  
      bigquery.SchemaField("Day_Number", bigquery.enums.SqlTypeNames.INTEGER),
      bigquery.SchemaField("Time", bigquery.enums.SqlTypeNames.TIME) 
  ],

  write_disposition = "WRITE_TRUNCATE",

)

In [ ]:
job = client.load_table_from_dataframe(
    date_dim_tbl2, table_id_date, job_config= job_config
)
job.result()

LoadJob<project=cis4400proj-370416, location=US, id=a0b88a82-7c0e-4d0b-852c-fc49944ef3cf>

# Loading Fact Table 

In [ ]:
# Client Object: How you read and write data into Google Big Query 
client = bigquery.Client()

In [ ]:
job_config = bigquery.LoadJobConfig(
    
  # Specify a partial schema. All columns are always written to the table
  # The schema is used to assist in data type definitions

  # Converting all the date values to string works 

  schema = [
      
      bigquery.SchemaField("Unique_Key", bigquery.enums.SqlTypeNames.INTEGER),
      bigquery.SchemaField("Agency_Dim_ID", bigquery.enums.SqlTypeNames.STRING),
      bigquery.SchemaField("Complaint_Type", bigquery.enums.SqlTypeNames.STRING),
      bigquery.SchemaField("Complaint_Channel", bigquery.enums.SqlTypeNames.STRING),
      bigquery.SchemaField("Complaint_Status", bigquery.enums.SqlTypeNames.STRING),
      bigquery.SchemaField("Date", bigquery.enums.SqlTypeNames.DATE),
      bigquery.SchemaField("Location_Dim_ID", bigquery.enums.SqlTypeNames.STRING)
  ],

  write_disposition = "WRITE_TRUNCATE",

)

In [ ]:
job = client.load_table_from_dataframe(
    complaint_fact_tbl, table_id_311_fact, job_config= job_config
)
job.result()

LoadJob<project=cis4400proj-370416, location=US, id=714fcb32-a23d-48f7-87b6-a76f2fcdb36c>